In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd
import numpy as np

In [2]:
#sentences = data["description"]
df = pd.read_csv("../data_pre/job data_pre(ver2).csv")
df

,job title,job requirements
0,Business Development Manager,"['role', 'cooperation', 'manage', 'director', ..."
1,Video and Motion Graphics Editor,"['join', 'organisation', 'difference', 'perfor..."
2,Programming Assistant - Casual,"['office', 'base', 'opportunity', 'gain', 'exp..."
3,QMC Duty Manager,"['amaze', 'organization', 'commit', 'community..."
4,Training Coordinator,"['opportunity', 'support', 'audio', 'language'..."
...,...,...
2349,Kitchen & Bath Cabinet Sales and Design,"['seek', 'design', 'sale', 'consultant', 'show..."
2350,Art Teacher - Trix Academy,"['art', 'teacher', 'scholar', 'deserve', 'qual..."
2351,CAMPUS ADJUNCT FACULTY- Foundations,"['faculty', 'foundation', 'campus', 'draw', 'c..."
2352,K-8 Art Teacher,"['school', 'school', 'network', 'charter', 'sc..."


In [3]:
content = df['job requirements']
sentences = []
for i in range(len(content)):
    a = content[i]
    a = a.replace(","," ")
    a = a.replace("'","")
    a = a.replace("[","")
    a = a.replace("]","")
    sentences.append(a)
#sentences

In [4]:
vectorizer = TfidfVectorizer(norm = None)
X = vectorizer.fit_transform(sentences)
print(X.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


# Ordering all words by tfidf

In [5]:
#print(vectorizer.vocabulary_)
#print(vectorizer.get_feature_names())
#print(X)
print(X.shape)
word = {"word":vectorizer.get_feature_names(), 'tfidf':X.toarray().sum(axis=0).tolist()}
word = pd.DataFrame(word)
word.sort_values(by='tfidf',ascending = False)

(2354, 11159)


/Users/caochuxue/opt/anaconda3/lib/python3.9/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


,word,tfidf
6685,museum,9659.387612
833,art,9437.041329
3886,experience,6706.650944
2984,design,6292.532551
9670,student,5409.217064
...,...,...
5315,interfacesability,8.071149
5313,interesse,8.071149
5312,interdisciplines,8.071149
5310,interdependent,8.071149


# words and tfidf for each document

In [6]:
# create a dictionary to store the word/key and tfidf/value
dict_of_tokens = {i[1]:i[0] for i in vectorizer.vocabulary_.items()}
tfidf_vectors = []
for row in X:
    tfidf_vectors.append({dict_of_tokens[column]:value for (column,value) in zip(row.indices,row.data)})


#words for document 0
print(tfidf_vectors[0])
#words for document 1
print(tfidf_vectors[1])

{'account': 3.4609910983914425, 'range': 2.7530288320463567, 'oversight': 5.026626388167149, 'culture': 2.8921782169751022, 'team': 3.3651748406898863, 'director': 2.719290692414506, 'manage': 2.429241754952459, 'cooperation': 5.719773568727095, 'role': 2.307526350878355}
{'opportunity': 2.057433669847771, 'progression': 5.153378093806293, 'career': 2.929485269387913, 'benefit': 2.4653467595945755, 'employee': 2.567851878655998, 'perform': 2.9928548833205024, 'difference': 4.045797135155423, 'organisation': 4.010705815344154, 'join': 2.917857231392794, 'team': 1.6825874203449431}


# Select the first 10 words as keywords

In [7]:
keyword_vectors = []
for i in range(len(tfidf_vectors)):
    document_i = tfidf_vectors[i]
    if len(document_i) <= 30:    #保留整个document的words
        keyword_vectors.append(document_i) 
    else:
        a = sorted(document_i.items(), key=lambda document_i: document_i[1], reverse=True)
        for j in range(len(a)-1,9,-1):  #从后往前删除
            #if a[j][1] != a[9][1]:
                a.pop(j)
        keyword_vectors.append(dict(a)) 
        

In [8]:
#key words for document 0
print(keyword_vectors[0])


#key words for document 1
print(keyword_vectors[1])

{'account': 3.4609910983914425, 'range': 2.7530288320463567, 'oversight': 5.026626388167149, 'culture': 2.8921782169751022, 'team': 3.3651748406898863, 'director': 2.719290692414506, 'manage': 2.429241754952459, 'cooperation': 5.719773568727095, 'role': 2.307526350878355}
{'opportunity': 2.057433669847771, 'progression': 5.153378093806293, 'career': 2.929485269387913, 'benefit': 2.4653467595945755, 'employee': 2.567851878655998, 'perform': 2.9928548833205024, 'difference': 4.045797135155423, 'organisation': 4.010705815344154, 'join': 2.917857231392794, 'team': 1.6825874203449431}


In [9]:
D = pd.DataFrame(keyword_vectors[0].keys())


for i in range(1,len(keyword_vectors)):
    df_b = pd.DataFrame(keyword_vectors[i].keys())
    D = pd.concat([D.reset_index(drop=True), df_b], axis=1)


In [10]:
D = D.T
D.to_excel("../data_key/keywords_job_30.xls", header = True, index= True)

/var/folders/lj/mv_hvlg946ng63pypv8qzf680000gn/T/ipykernel_30735/3986824649.py:2: FutureWarning: As the xlwt package is no longer maintained, the xlwt engine will be removed in a future version of pandas. This is the only engine in pandas that supports writing in the xls format. Install openpyxl and write to an xlsx file instead. You can set the option io.excel.xls.writer to 'xlwt' to silence this warning. While this option is deprecated and will also raise a warning, it can be globally set and the warning suppressed.
  D.to_excel("../data_key/keywords_job_30.xls", header = True, index= True)


# Topic finding using LDA

In [11]:
from gensim import corpora, models

In [12]:
# list_of_list_of_tokens = [["a","b","c"], ["d","e","f"]]
# ["a","b","c"] are the tokens of document 1, ["d","e","f"] are the tokens of document 2...
content = df['job requirements']
list_of_list_of_tokens = []
for i in range(len(content)):
    a = content[i]
    a = a.replace(","," ")
    a = a.replace("'","")
    a = a.replace("[","")
    a = a.replace("]","")
    list_of_list_of_tokens.append(a.split(" "))

In [13]:
dictionary_LDA = corpora.Dictionary(list_of_list_of_tokens)
dictionary_LDA.filter_extremes(no_below=3)
corpus = [dictionary_LDA.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens]

num_topics = 2 #can change this number of topic
lda_model = models.LdaModel(corpus, num_topics=num_topics, 
                            id2word=dictionary_LDA,                                   
                            passes=4, alpha=[0.01]*num_topics,
                            eta=[0.01]*len(dictionary_LDA.keys()))

In [14]:
#print topic found:
for i,topic in lda_model.show_topics(formatted=True, num_topics=num_topics, num_words=30):
    print(str(i)+": "+ topic)
    print()

0: 0.020*"museum" + 0.020*"art" + 0.010*"program" + 0.008*"position" + 0.008*"support" + 0.007*"ability" + 0.007*"staff" + 0.007*"require" + 0.007*"community" + 0.006*"event" + 0.006*"provide" + 0.006*"time" + 0.006*"employee" + 0.005*"director" + 0.005*"communication" + 0.005*"job" + 0.005*"development" + 0.005*"opportunity" + 0.005*"project" + 0.004*"collection" + 0.004*"status" + 0.004*"department" + 0.004*"employment" + 0.004*"management" + 0.004*"environment" + 0.004*"exhibition" + 0.004*"application" + 0.004*"manage" + 0.004*"benefit" + 0.004*"report"

1: 0.017*"art" + 0.015*"student" + 0.015*"design" + 0.010*"ability" + 0.008*"school" + 0.007*"position" + 0.007*"require" + 0.006*"time" + 0.006*"job" + 0.006*"program" + 0.006*"project" + 0.006*"provide" + 0.005*"teach" + 0.005*"knowledge" + 0.005*"develop" + 0.005*"environment" + 0.005*"create" + 0.005*"education" + 0.005*"opportunity" + 0.005*"client" + 0.004*"community" + 0.004*"management" + 0.004*"development" + 0.004*"employ

In [15]:
#To print the % of topics a document is about, do the following:
#(14, 0.9983065953654187)means the first document is 99.8% about topic 14.

lda_model[corpus[0]] # corpus[0] means the first document.

[(0, 0.99875283)]

# Repeat the process for job dataset version 2

In [16]:
#sentences = data["description"]
df2 = pd.read_csv("/Users/as/Desktop/job data_pre(ver2).csv")
df2

FileNotFoundError: [Errno 2] No such file or directory: '/Users/as/Desktop/job data_pre(ver2).csv'

In [ ]:
content2 = df2['job requirements']
sentences2 = []
for i in range(len(content2)):
    a = content2[i]
    a = a.replace(","," ")
    a = a.replace("'","")
    a = a.replace("[","")
    a = a.replace("]","")
    sentences2.append(a)
#sentences

In [ ]:
vectorizer2 = TfidfVectorizer(norm = None)
X2 = vectorizer2.fit_transform(sentences2)

In [ ]:
print(X2.shape)
word2 = {"word":vectorizer2.get_feature_names(), 'tfidf':X2.toarray().sum(axis=0).tolist()}
word2 = pd.DataFrame(word2)
word2.sort_values(by='tfidf',ascending = False)

In [ ]:
# create a dictionary to store the word/key and tfidf/value
dict_of_tokens2 = {i[1]:i[0] for i in vectorizer2.vocabulary_.items()}
tfidf_vectors2 = []
for row in X2:
    tfidf_vectors2.append({dict_of_tokens2[column]:value for (column,value) in zip(row.indices,row.data)})


#words for document 0
print(tfidf_vectors2[0])
#words for document 1
print(tfidf_vectors2[1])

# Select the first 10 words as keywords

In [ ]:
keyword_vectors2 = []
for i in range(len(tfidf_vectors2)):
    document_i = tfidf_vectors2[i]
    if len(document_i.keys()) <= 10:    #保留整个document的words
        keyword_vectors2.append(document_i) 
    else:
        a = sorted(document_i.items(), key=lambda document_i: document_i[1], reverse=True)
        for j in range(len(a)-1,9,-1):  #从后往前删除
            #if a[j][1] != a[9][1]:
                a.pop(j)
        keyword_vectors2.append(dict(a)) 
        

In [ ]:
#key words for document 0
print(keyword_vectors2[0])


#key words for document 1
print(keyword_vectors2[1])

In [ ]:
D2 = pd.DataFrame(keyword_vectors2[0].keys())


for i in range(1,len(keyword_vectors2)):
    df_b = pd.DataFrame(keyword_vectors2[i].keys())
    D2 = pd.concat([D2.reset_index(drop=True), df_b], axis=1)

    
D2 = D2.T
D2.to_excel("keywords job(ver2).xls", header = True, index= True)

# Topic finding

In [ ]:
# list_of_list_of_tokens = [["a","b","c"], ["d","e","f"]]
# ["a","b","c"] are the tokens of document 1, ["d","e","f"] are the tokens of document 2...
content2 = df2['job requirements']
list_of_list_of_tokens2 = []
for i in range(len(content2)):
    a = content2[i]
    a = a.replace(","," ")
    a = a.replace("'","")
    a = a.replace("[","")
    a = a.replace("]","")
    list_of_list_of_tokens2.append(a.split(" "))

In [ ]:
dictionary_LDA2 = corpora.Dictionary(list_of_list_of_tokens2)
dictionary_LDA2.filter_extremes(no_below=3)
corpus2 = [dictionary_LDA2.doc2bow(list_of_tokens) for list_of_tokens in list_of_list_of_tokens2]

num_topics = 2 #can change this number of topic
lda_model2 = models.LdaModel(corpus2, num_topics=num_topics, 
                            id2word=dictionary_LDA2,                                   
                            passes=4, alpha=[0.01]*num_topics,
                            eta=[0.01]*len(dictionary_LDA2.keys()))

In [ ]:
#print topic found:
for i,topic in lda_model2.show_topics(formatted=True, num_topics=num_topics, num_words=10):
    print(str(i)+": "+ topic)
    print()

In [ ]:
#To print the % of topics a document is about, do the following:
#(14, 0.9983065953654187)means the first document is 99.8% about topic 14.

lda_model2[corpus2[0]] # corpus[0] means the first document.